In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime as dt
import cv2
import keras
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [111]:
data_dir = './' #Change to your directory here

def load_data(data_dir):
    train = pd.read_json(data_dir + 'train.json')
    print("Train data loaded")
    test = pd.read_json(data_dir + 'test.json')
    print("Test data loaded")
    #Fill 'na' angles with mode
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    test.inc_angle = test.inc_angle.replace('na', 0)
    test.inc_angle = test.inc_angle.astype(float).fillna(0.0)
    return train, test

train, test = load_data(data_dir)
print("done")

X_angle_train = np.array(train.inc_angle)
X_angle_test = np.array(test.inc_angle)


def color_composite(data):
    w,h = 75,75
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        band_4 = band_2 * row['inc_angle']


        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))
        a = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))

        
        rgba = np.dstack((r, g, b))

        #rgba = cv2.resize(rgba, (w,h)).astype(np.float32)
        
        rgb_arrays.append(rgba)
    return np.array(rgb_arrays)

rgb_train = color_composite(train)
rgb_test = color_composite(test)
print("RGBs Done")

y_train = np.array(train['is_iceberg'])

Train data loaded
Test data loaded
done
RGBs Done


# Models

In [112]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model

def conv_block(x, nf=8, k=3, s=1, nb=2, p_act='elu'):
    
    for i in range(nb):
        x = Conv2D(filters=nf, kernel_size=(k, k), strides=(s, s),  
                   padding='same', kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        x = keras.layers.LeakyReLU()(x)
        
    return x

def res_block(x, nf=64, k=3, s=1, nb=1):
    
    for i in range(nb):
        x = Conv2D(filters=nf, kernel_size=(k, k), strides=(s, s),  
                   padding='same', kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        x = keras.layers.LeakyReLU()(x)
        x = Dropout(0.2)(x)
        nf*=2
        x = Conv2D(filters=nf, kernel_size=(k, k), strides=(s, s),  
                   padding='same', kernel_initializer='he_uniform')(x)
        x = BatchNormalization()(x)
        x = keras.layers.LeakyReLU()(x)
        
    return x

def dense_block(x, h=32, d=0.5, m=0., p_act='elu'):
    return Dropout(d) (BatchNormalization(momentum=m) (Dense(h, activation=p_act)(x)))


def bn_pooling(x, k=2, s=2, m=0): 
    return MaxPooling2D((k, k), strides=(s, s))(x)

In [125]:
def get_simple_model_var1():
    
    bn_model = 0.99
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (5,5), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (5,5), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(256, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.1)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_ayer = Dropout(0.4) (BatchNormalization(momentum=bn_model) ( Dense(1024, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.4) (BatchNormalization(momentum=bn_model) ( Dense(1024, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

def get_simple_model():
    bn_model = 0.99
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 2), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.1)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.1)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_ayer = Dropout(0.4) (BatchNormalization(momentum=bn_model) ( Dense(512, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.4) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

def get_model_with_res(img_shape=(75, 75, 2), num_classes=1, f=8, h=512):
     
    #model
    bn_model = 0.99
    p_activation = 'elu'
    
    #
    input_img = Input(shape=img_shape, name='img_inputs')
    input_img_bn = BatchNormalization(momentum=bn_model)(input_img)
    
    #
    input_meta = Input(shape=[1], name='angle')
    input_meta_bn = BatchNormalization(momentum=bn_model)(input_meta)

    
    #img_1
    #img_1:block_1  8
    f=32
    img_1 = conv_block(input_img_bn, nf=f, k=3, s=1, nb=1,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=2, m=bn_model)
    
    #img_1:block_2
    f=64
    img_1 = Dropout(0.2)(img_1)
    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=1,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=2, m=bn_model) 
    
    #img_1:block_2
    f=64
    img_1 = res_block(img_1, nf=f, k=3, s=1, nb=3)

    
    #img_1:block_3
    f=128

    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=1,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=3, m=bn_model)
    img_1 = Dropout(0.2)(img_1)
    f=256
    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=1,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=3, m=bn_model)
    img_1 = Dropout(0.2)(img_1)
    f=512
    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=1,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=3, m=bn_model)
    img_1 = Dropout(0.2)(img_1)    
    img_1 = GlobalMaxPooling2D()(img_1)
    
    #full connect
    concat = (Concatenate()([img_1, input_meta_bn]))
    x = dense_block(img_1, h=h)
    x = dense_block(x, h=256)
    output = Dense(num_classes, activation='sigmoid')(x)
    
    model = Model([input_img, input_meta], output)
    
    return model

def get_model(img_shape=(75, 75, 2), num_classes=1, f=8, h=512):

    """
    img_shape: dimension for input image
    f: filters of first conv blocks and generate filters in the following 
       blocks acorrdingly 
    h: units in dense hidden layer
    """ 
    
    #model
    bn_model = 0.99
    p_activation = 'elu'
    
    #
    input_img = Input(shape=img_shape, name='img_inputs')
    input_img_bn = BatchNormalization(momentum=bn_model)(input_img)
    
    #
    input_meta = Input(shape=[1], name='angle')
    input_meta_bn = BatchNormalization(momentum=bn_model)(input_meta)

    
    #img_1
    #img_1:block_1  8
    img_1 = conv_block(input_img_bn, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=2, m=bn_model)
    
    #img_1:block_2
    f=16
    img_1 = Dropout(0.2)(img_1)
    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=2, m=bn_model) 
    
    #img_1:block_3
    f=32
    img_1 = Dropout(0.2)(img_1)
    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_1 = bn_pooling(img_1, k=2, s=3, m=bn_model)
    
    #img_1:block_4
    f=64
    img_1 = Dropout(0.2)(img_1)
    img_1 = conv_block(img_1, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_1 = Dropout(0.2)(img_1)
    img_1 = BatchNormalization(momentum=bn_model)(GlobalMaxPooling2D()(img_1))
    
    #img 2 #32  | tot_layers: 6
    f = 8
    img_2 = conv_block(input_img_bn, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_2 = bn_pooling(img_2, k=2, s=3, m=bn_model)
    
    #img_1:block_2
    f=16
    img_2 = Dropout(0.2)(img_2)
    img_2 = conv_block(img_2, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_2 = bn_pooling(img_2, k=2, s=2, m=bn_model) 
    
    #img_1:block_3
    f=32
    img_2 = Dropout(0.2)(img_2)
    img_2 = conv_block(img_2, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_2 = bn_pooling(img_2, k=2, s=3, m=bn_model)
    
    #img_1:block_4
    f=64
    img_2 = Dropout(0.2)(img_2)
    img_2 = conv_block(img_2, nf=f, k=3, s=1, nb=6,p_act=p_activation)
    img_2 = Dropout(0.2)(img_2)
    img_2 = BatchNormalization(momentum=bn_model)(GlobalMaxPooling2D()(img_2))
    
    
    #full connect
    concat = (Concatenate()([img_1, img_2, input_meta_bn]))
    x = dense_block(img_1, h=h)
    x = dense_block(x, h=h)
    output = Dense(num_classes, activation='sigmoid')(x)
    
    model = Model([input_img, input_meta], output)
    
    return model

In [114]:
from sklearn.model_selection import train_test_split

X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(rgb_train,
                                    X_angle_train, y_train, random_state=420, test_size=0.25)

print("X_train:",X_train.shape)
print("X_valid:",X_valid.shape)

X_train: (1203, 75, 75, 3)
X_valid: (401, 75, 75, 3)


# Current model

In [123]:
from keras import backend as K
K.clear_session()

#model = get_model(img_shape=(75,75,2))
#model = get_model_with_res(img_shape=(75,75,2))
#model = get_simple_model()
model = get_simple_model_var1()


print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_1 (InputLayer)                (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 75, 75, 3)    12          X_1[0][0]                        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 71, 71, 16)   1216        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 67, 67, 16)   6416        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [124]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 32
#Lets define the image transormations that we want
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1,
                         rotation_range=90)

# Here is the function that merges our two generators
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=666)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=666)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
gen_flow = gen_flow_for_two_inputs(X_train, X_angle_train, y_train)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
#call backs
weights_file = './model.h5'

earlystop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, min_delta=1e-4, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=40, verbose=1, epsilon=1e-4, mode='min')
model_chk = ModelCheckpoint(monitor='val_loss', filepath=weights_file, save_best_only=True, 
                            save_weights_only=True, mode='min')
        
callbacks = [earlystop, reduce_lr_loss, model_chk, TensorBoard(log_dir='./logs1')]

model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.002), 
              loss='binary_crossentropy', metrics=['accuracy'])

#fit the model
model.fit_generator(gen_flow, validation_data=([X_valid, X_angle_valid], y_valid), 
                    steps_per_epoch=np.ceil(8 * float(len(y_train)) / float(batch_size)), 
                    epochs=500, verbose=1, callbacks=callbacks)
#model.fit([X_train, X_angle_train],y_train, validation_data=([X_valid, X_angle_valid], y_valid),  
 #                   epochs=500, verbose=1, callbacks=callbacks)


Epoch 1/500
301/301 [==============================] - 28s 94ms/step - loss: 0.5409 - acc: 0.7783 - val_loss: 0.5363 - val_acc: 0.8080
Epoch 2/500
301/301 [==============================] - 24s 79ms/step - loss: 0.3920 - acc: 0.8271 - val_loss: 0.5621 - val_acc: 0.8130
Epoch 3/500
301/301 [==============================] - 25s 84ms/step - loss: 0.3410 - acc: 0.8502 - val_loss: 0.2708 - val_acc: 0.8903
Epoch 4/500
301/301 [==============================] - 27s 89ms/step - loss: 0.3295 - acc: 0.8560 - val_loss: 0.4204 - val_acc: 0.8080
Epoch 5/500
301/301 [==============================] - 26s 87ms/step - loss: 0.3168 - acc: 0.8636 - val_loss: 0.2470 - val_acc: 0.8953
Epoch 6/500
301/301 [==============================] - 25s 83ms/step - loss: 0.3045 - acc: 0.8654 - val_loss: 0.3356 - val_acc: 0.8828
Epoch 7/500
301/301 [==============================] - 25s 83ms/step - loss: 0.2939 - acc: 0.8716 - val_loss: 0.2799 - val_acc: 0.8828
Epoch 8/500
301/301 [==============================] - 

301/301 [==============================] - 23s 77ms/step - loss: 0.2075 - acc: 0.9138 - val_loss: 0.2304 - val_acc: 0.9077
Epoch 62/500
301/301 [==============================] - 23s 77ms/step - loss: 0.2055 - acc: 0.9127 - val_loss: 0.2619 - val_acc: 0.9052
Epoch 63/500
301/301 [==============================] - 23s 77ms/step - loss: 0.2019 - acc: 0.9171 - val_loss: 0.2446 - val_acc: 0.9052
Epoch 64/500
301/301 [==============================] - 23s 77ms/step - loss: 0.2142 - acc: 0.9113 - val_loss: 0.2489 - val_acc: 0.9002
Epoch 65/500
301/301 [==============================] - 23s 77ms/step - loss: 0.2019 - acc: 0.9137 - val_loss: 0.2551 - val_acc: 0.8978
Epoch 66/500
301/301 [==============================] - 23s 78ms/step - loss: 0.2006 - acc: 0.9150 - val_loss: 0.2370 - val_acc: 0.9052
Epoch 67/500
301/301 [==============================] - 23s 77ms/step - loss: 0.2014 - acc: 0.9165 - val_loss: 0.2484 - val_acc: 0.9052
Epoch 68/500
300/301 [============================>.] - ETA: 

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss

#train, validataion split
test_ratio = 0.159
nr_runs = 5
split_seed = 25
epochs = 1000
kf = StratifiedShuffleSplit(n_splits=nr_runs, test_size=test_ratio, train_size=None, random_state=split_seed)

train_y = train['is_iceberg'].values
split_indices = train_y.copy()

train_X = rgb_train
train_meta = train['inc_angle'].values

#training, evaluation, test and make submission
for r, (train_index, valid_index) in enumerate(kf.split(train, split_indices)):
    tmp = dt.datetime.now().strftime("%Y-%m-%d-%H-%M")
        
    y1, y2 = train_y[train_index], train_y[valid_index]
    x1, x2 = train_X[train_index], train_X[valid_index]
    xm1, xm2 = train_meta[train_index], train_meta[valid_index]

    print('splitted: {0}, {1}'.format(x1.shape, x2.shape), flush=True)
    print('splitted: {0}, {1}'.format(y1.shape, y2.shape), flush=True)
    ################################
    if r > 0:
        model.load_weights('model.h5')
        
    #optim = SGD(lr=0.005, momentum=0.0, decay=0.002, nesterov=True)
    optim = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.002)
        
    model.compile(optimizer=optim, loss="binary_crossentropy", metrics=["accuracy"])
    #call backs
    earlystop = EarlyStopping(monitor='val_loss', patience=100, verbose=1, min_delta=1e-4, mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=40, verbose=1, epsilon=1e-4, mode='min')
    model_chk = ModelCheckpoint(monitor='val_loss', filepath='model.h5', save_best_only=True, 
                                save_weights_only=True, mode='min')
        
    callbacks = [earlystop, reduce_lr_loss, model_chk, TensorBoard(log_dir='./logs_cv')]
        ##########
       
    model.fit_generator(generator=gen_flow_for_two_inputs(x1, xm1, y1),
                        steps_per_epoch= np.ceil(12 * float(len(y1)) / float(batch_size)),
                        epochs=epochs,
                        verbose=2,
                        callbacks=callbacks,
                        validation_data=([x2, xm2], y2))

     

    model.load_weights('model.h5')
            
    p = model.predict([x2, xm2], batch_size=batch_size, verbose=1)
    print('\n\nEvaluate loss in validation data: {}'.format(log_loss(y2, p)), flush=True)

    p = model.predict([x1, xm1], batch_size=batch_size, verbose=1)
    print('\n\nEvaluate loss in training data: {}'.format(log_loss(y1, p)), flush=True)
           
    print('\nPredict...', flush=True)
    ids = test['id'].values

    #prediction
    pred = model.predict([test_X_dup, test_meta], batch_size=batch_size, verbose=1)
    pred = np.squeeze(pred, axis=-1)
            
    file = 'subm_{}_f{:03d}.csv'.format(tmp, nb_filters)
    subm = pd.DataFrame({'id': ids, target: pred})
    subm.to_csv('./submitions/{}'.format(file), index=False, float_format='%.6f')


splitted: (1348, 199, 199, 2), (256, 199, 199, 2)
splitted: (1348,), (256,)


C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\keras\preprocessing\image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1348, 199, 199, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/1000
 - 227s - loss: 0.7768 - acc: 0.6080 - val_loss: 0.5501 - val_acc: 0.6562
Epoch 2/1000
 - 223s - loss: 0.6647 - acc: 0.6359 - val_loss: 0.8509 - val_acc: 0.6094
Epoch 3/1000
 - 222s - loss: 0.6292 - acc: 0.6624 - val_loss: 0.6694 - val_acc: 0.7266
Epoch 4/1000
 - 222s - loss: 0.5671 - acc: 0.7160 - val_loss: 0.5135 - val_acc: 0.7695
Epoch 5/1000
 - 223s - loss: 0.5097 - acc: 0.7713 - val_loss: 0.4972 - val_acc: 0.7852
Epoch 6/1000
 - 223s - loss: 0.4549 - acc: 0.7917 - val_loss: 0.4711 - val_acc: 0.8008
Epoch 7/1000
 - 222s - loss: 0.4221 - acc: 0.8051 - val_loss: 0.4220 - val_acc: 0.8125
Epoch 8/1000
 - 222s - loss: 0.4232 - acc: 0.8059 - val_loss: 0.4748 - val_acc: 0.7852
Epoch 9/1000
 - 223s - loss: 0.4080 - acc: 0.8081 - val_loss: 0.3783 - val_acc: 0.8281
Epoch 10/1000
 - 222s - loss: 0.3845 - acc: 0.8217 - val_loss: 0.4678 - val_acc: 0.7930
Epoch 11/1000
 - 222s - loss: 0.3867 - acc: 0.8217 - val_loss: 0.4340 - val_acc: 0.7969
Epoch 12/1000
 - 222s - loss: 0.3755 - ac

In [64]:
# Predict on test data
from keras.models import load_model
model.load_weights('model.h5')

print(model.evaluate([X_train,X_angle_train], y_train))
print(model.evaluate([X_valid,X_angle_valid], y_valid))

test_predictions = model.predict([rgb_test, X_angle_test])

tmp = dt.datetime.now().strftime("%Y-%m-%d-%H-%M")
nb_filters = 2
file = 'subm_{}_f{:03d}.csv'.format(tmp, nb_filters)

# Create .csv
pred_df = test[['id']].copy()
pred_df['is_iceberg'] = test_predictions
pred_df.to_csv('./submitions/{}'.format(file), index=False, float_format='%.6f')
pred_df.head()

1363/1363 [==============================] - 1s 957us/step
[0.20873701845733716, 0.90755685995539936]
241/241 [==============================] - 0s 1ms/step
[0.14861328541243224, 0.94605809153362919]


,id,is_iceberg
0,5941774d,0.383842
1,4023181e,0.047591
2,b20200e4,0.035624
3,e7f018bb,0.986744
4,4371c8c3,0.069155
